In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'direction_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${sp

In [4]:
analysis_date

'2024-10-16'

In [5]:
# path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [6]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [7]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_speedmap_segments_detail_2024-10-16.parquet'

In [8]:
detail = gpd.read_parquet(path)

In [9]:
detail >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch
0,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b__5b9a7352...,Woodlake Park EB__Cypress St & Naranjo Blvd WB,0d1ef8b5-90e4-40de-b37b-504cb862d18b-5b9a7352-...,AM Peak,17.27,...,15.20,20.96,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.09762 36.41481, -119.09762 36...",4,1.333333
1,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,5b9a7352-556f-4272-b663-22f9821d6a40__5f894da7...,Cypress St & Naranjo Blvd WB__Millwood and Roa...,5b9a7352-556f-4272-b663-22f9821d6a40-5f894da7-...,AM Peak,49.50,...,43.85,51.81,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.10550 36.41377, -119.10562 36...",4,1.333333
2,0139b1253130b33adcd4b3a4490530d2,51530f7b5ba4cdde7c644c14f262005c,b0c096a2-4c32-4675-b005-43b58b3781e6,874b0dfa-ab3d-47da-ac75-5fd7b9bdea42,C30 WB,5b9a7352-556f-4272-b663-22f9821d6a40__5f894da7...,Cypress St & Naranjo Blvd WB__Millwood and Roa...,5b9a7352-556f-4272-b663-22f9821d6a40-5f894da7-...,AM Peak,58.87,...,57.15,59.50,TCRTA TripShot Schedule,06 - Fresno,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,"LINESTRING (-119.11666 36.41379, -119.11684 36...",4,1.333333


In [10]:
path = f'{catalog.schedule_downloads.dir}{catalog.schedule_downloads.trips}_{analysis_date}.parquet'

In [11]:
path

'gs://calitp-analytics-data/data-analyses/rt_delay/compiled_cached_views/trips_2024-10-16.parquet'

In [12]:
trips = pd.read_parquet(path)

In [13]:
df = gtfs_schedule_wrangling.get_sched_trips_hr(analysis_date)

In [14]:
df >> head(3)

,route_id,shape_id,time_of_day,schedule_gtfs_dataset_key,n_trips,trips_hr
0,001,0010111,AM Peak,9809d3f8121513057bc5cb8de7b54ce2,3,1.000000
1,001,0010111,Early AM,9809d3f8121513057bc5cb8de7b54ce2,1,0.333333
2,001,0010111,Midday,9809d3f8121513057bc5cb8de7b54ce2,5,1.000000


In [15]:
gtfs_schedule_wrangling.merge_route_identifiers(df, analysis_date)

,route_id,shape_id,time_of_day,schedule_gtfs_dataset_key,n_trips,trips_hr,route_short_name
0,001,0010111,AM Peak,9809d3f8121513057bc5cb8de7b54ce2,3,1.000000,1
1,001,0010111,Early AM,9809d3f8121513057bc5cb8de7b54ce2,1,0.333333,1
2,001,0010111,Midday,9809d3f8121513057bc5cb8de7b54ce2,5,1.000000,1
3,001,0010111,PM Peak,9809d3f8121513057bc5cb8de7b54ce2,3,0.600000,1
4,001,0010113,AM Peak,9809d3f8121513057bc5cb8de7b54ce2,3,1.000000,1
...,...,...,...,...,...,...,...
19176,shrdserv,shrdserv_loop_3,PM Peak,a8d5f90bfd689badb7e1deb041408e96,1,0.200000,Shared Services
19177,valleywide,valleywide,AM Peak,31152914d10e2d0977b8b2fabb167922,12,4.000000,None
19178,valleywide,valleywide,Evening,31152914d10e2d0977b8b2fabb167922,9,1.800000,None
19179,valleywide,valleywide,Midday,31152914d10e2d0977b8b2fabb167922,20,4.000000,None


## Where'd the J line go??

In [16]:
df >> filter(_.route_id.str.contains('910'))

,route_id,shape_id,time_of_day,schedule_gtfs_dataset_key,n_trips,trips_hr
16242,910-13183,9100209_JUNE24,AM Peak,0666caf3ec1ecc96b74f4477ee4bc939,7,2.333333
16243,910-13183,9100209_JUNE24,Early AM,0666caf3ec1ecc96b74f4477ee4bc939,8,2.666667
16244,910-13183,9100209_JUNE24,Midday,0666caf3ec1ecc96b74f4477ee4bc939,12,2.400000
16245,910-13183,9100209_JUNE24,Owl,0666caf3ec1ecc96b74f4477ee4bc939,1,0.250000
16246,910-13183,9100209_JUNE24,PM Peak,0666caf3ec1ecc96b74f4477ee4bc939,15,3.000000
16247,910-13183,9100210_JUNE24,AM Peak,0666caf3ec1ecc96b74f4477ee4bc939,19,6.333333
16248,910-13183,9100210_JUNE24,Early AM,0666caf3ec1ecc96b74f4477ee4bc939,18,6.000000
16249,910-13183,9100210_JUNE24,Evening,0666caf3ec1ecc96b74f4477ee4bc939,11,2.200000
16250,910-13183,9100210_JUNE24,Midday,0666caf3ec1ecc96b74f4477ee4bc939,21,4.200000
16251,910-13183,9100210_JUNE24,Owl,0666caf3ec1ecc96b74f4477ee4bc939,4,1.000000


In [16]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'direction_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${sp

In [17]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [41]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.route_dir_single_segment}_{analysis_date}.parquet'

In [ ]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [ ]:
speedmap_segs >> filter(_.route_id.str.contains('910'))

it has no route_short_name...
```
857-13183,857,Metro Local Line,C & K LINE LINK,3,,,
901-13183,,Metro G Line 901,METRO G LINE,3,FC4C02,FFFFFF,https://www.metro.net/g-line/
910-13183,,Metro J Line 910/950,METRO J LINE,3,ADB8BF,000000,https://www.metro.net/j-line/
```